In [1]:
# Install required packages
# !pip install dspy-ai litellm mlflow python-dotenv

import dspy
import mlflow
import os
from dotenv import load_dotenv
from typing import List

print("✅ All imports successful!")

✅ All imports successful!


In [2]:
# Load environment variables
load_dotenv()

# Set up MLflow experiment
mlflow.set_experiment("DSPy-Optimizer-Groq")

# Enable comprehensive tracing for optimization
mlflow.dspy.autolog(
    log_evals=True,           # Log evaluation results
    log_compiles=True,        # Log optimization process
    log_traces_from_compile=True  # Trace during optimization
)

print("🔍 MLflow tracing enabled with optimization tracking!")
print(f"📊 Experiment: DSPy-Optimizer-Groq")


2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/04 14:17:50 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/04 14:17:50 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/04 14:17:50 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/04 14:17:50 INFO mlflow.tracking.fluent: Experiment with name 'DSPy-Optimizer-Groq' does not exist. Creating a new experiment.


🔍 MLflow tracing enabled with optimization tracking!
📊 Experiment: DSPy-Optimizer-Groq


In [3]:
# Configure Groq with Llama 3.3 70B Versatile
lm = dspy.LM(
    'groq/llama-3.3-70b-versatile',
    api_key=os.getenv('GROQ_API_KEY'),
    max_tokens=1024,
    temperature=0.7
)

# Set as default LM
dspy.configure(lm=lm)

print("🚀 Groq (Llama 3.3 70B Versatile) configured!")
print(f"Model: {lm.model}")
print(f"📊 Free tier limits:")
print(f"   - Requests: 30/min, 1K/day")
print(f"   - Tokens: 12K/min, 100K/day")

# Quick test
test = dspy.Predict("question -> answer")
result = test(question="What is the capital of France?")
print(f"\nTest: Capital of France = {result.answer}")
print("✅ Groq connection verified!")


🚀 Groq (Llama 3.3 70B Versatile) configured!
Model: groq/llama-3.3-70b-versatile
📊 Free tier limits:
   - Requests: 30/min, 1K/day
   - Tokens: 12K/min, 100K/day

Test: Capital of France = The capital of France is Paris.
✅ Groq connection verified!


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## an...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(


In [4]:
# Mock Wikipedia database (since public ColBERT servers are often down)
WIKIPEDIA_MOCK = {
    "python programming": [
        "Python is a high-level programming language created by Guido van Rossum in 1991.",
        "Python emphasizes code readability with significant whitespace and supports multiple programming paradigms.",
        "Python is widely used in web development, data science, and machine learning."
    ],
    "machine learning": [
        "Machine learning is a subset of artificial intelligence that enables systems to learn from data.",
        "Common machine learning algorithms include decision trees, neural networks, and support vector machines.",
        "Machine learning is used in recommendation systems, image recognition, and natural language processing."
    ],
    "eiffel tower": [
        "The Eiffel Tower is located in Paris, France and was completed in 1889.",
        "The tower stands 330 meters tall and was designed by Gustave Eiffel.",
        "It was initially criticized but is now one of the most visited monuments in the world."
    ],
    "albert einstein": [
        "Albert Einstein was a theoretical physicist born in 1879 in Germany.",
        "He developed the theory of relativity, with E=mc² being his most famous equation.",
        "Einstein received the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect."
    ]
}

def search_wikipedia(query: str) -> List[str]:
    """
    Search Wikipedia for relevant passages.
    Returns top 3 most relevant text passages.
    
    Args:
        query: Search query string
    
    Returns:
        List of relevant text passages from Wikipedia
    """
    query_lower = query.lower()
    
    # Simple keyword matching
    results = []
    for topic, passages in WIKIPEDIA_MOCK.items():
        if any(word in query_lower for word in topic.split()):
            results.extend(passages)
    
    # If no matches, return general info
    if not results:
        results = ["No specific information found. Please try a more specific query."]
    
    return results[:3]  # Return top 3 results


print("📚 Mock Wikipedia search tool created")
print("\n🔍 Test search:")
test_results = search_wikipedia("What is Python?")
for i, result in enumerate(test_results, 1):
    print(f"  {i}. {result[:80]}...")


📚 Mock Wikipedia search tool created

🔍 Test search:
  1. Python is a high-level programming language created by Guido van Rossum in 1991....
  2. Python emphasizes code readability with significant whitespace and supports mult...
  3. Python is widely used in web development, data science, and machine learning....


In [5]:
# Create ReAct agent with Wikipedia search tool
react_agent = dspy.ReAct(
    signature="question -> answer",
    tools=[search_wikipedia],
    max_iters=5
)

print("🤖 ReAct RAG Agent created")
print("   - Signature: question -> answer")
print("   - Tools: search_wikipedia")
print("   - Max iterations: 5")


🤖 ReAct RAG Agent created
   - Signature: question -> answer
   - Tools: search_wikipedia
   - Max iterations: 5


In [6]:
# Create training dataset (small dataset is sufficient for DSPy)
trainset = [
    dspy.Example(
        question="What programming language was created by Guido van Rossum?",
        answer="Python"
    ).with_inputs("question"),
    
    dspy.Example(
        question="When was Python first released?",
        answer="1991"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What is a subset of AI that learns from data?",
        answer="Machine learning"
    ).with_inputs("question"),
    
    dspy.Example(
        question="Where is the Eiffel Tower located?",
        answer="Paris"
    ).with_inputs("question"),
    
    dspy.Example(
        question="Who designed the Eiffel Tower?",
        answer="Gustave Eiffel"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What year was Albert Einstein born?",
        answer="1879"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What is Einstein's famous equation?",
        answer="E=mc²"
    ).with_inputs("question"),
]

# Create validation dataset
valset = [
    dspy.Example(
        question="What does Python emphasize in code?",
        answer="readability"
    ).with_inputs("question"),
    
    dspy.Example(
        question="When was the Eiffel Tower completed?",
        answer="1889"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What prize did Einstein win in 1921?",
        answer="Nobel Prize in Physics"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What are common machine learning algorithms?",
        answer="decision trees"
    ).with_inputs("question"),
]

print(f"📊 Datasets created:")
print(f"   - Training set: {len(trainset)} examples")
print(f"   - Validation set: {len(valset)} examples")
print(f"\n📝 Sample training example:")
print(f"   Question: {trainset[0].question}")
print(f"   Answer: {trainset[0].answer}")


📊 Datasets created:
   - Training set: 7 examples
   - Validation set: 4 examples

📝 Sample training example:
   Question: What programming language was created by Guido van Rossum?
   Answer: Python


In [7]:
print("🧪 Testing baseline agent (before optimization)\n")

# Test on a few examples
test_questions = [
    "What programming language was created by Guido van Rossum?",
    "Where is the Eiffel Tower located?",
    "What is Einstein's famous equation?"
]

for question in test_questions:
    result = react_agent(question=question)
    print(f"Q: {question}")
    print(f"A: {result.answer}")
    print("-" * 60)


🧪 Testing baseline agent (before optimization)



d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=

Q: What programming language was created by Guido van Rossum?
A: Python
------------------------------------------------------------


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=

Q: Where is the Eiffel Tower located?
A: The Eiffel Tower is located in Paris, France.
------------------------------------------------------------


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ne...er_specific_fields=None), input_type=

Q: What is Einstein's famous equation?
A: E=mc^2
------------------------------------------------------------


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(


In [8]:
# Use built-in exact match metric with flexibility
def custom_metric(example, pred, trace=None):
    """
    Custom metric that checks if prediction matches answer.
    Allows partial matches for better optimization.
    """
    # Use DSPy's built-in answer matching with partial match (F1 > 0.5)
    return dspy.evaluate.answer_exact_match(
        example, 
        pred, 
        trace=trace,
        frac=0.5  # Allow 50% F1 match (more lenient than exact match)
    )

print("✅ Evaluation metric defined: custom_metric (F1 >= 0.5)")


✅ Evaluation metric defined: custom_metric (F1 >= 0.5)


In [9]:
# Initialize MIPROv2 optimizer
optimizer = dspy.MIPROv2(
    metric=custom_metric,
    auto="light",  # Options: "light", "medium", "heavy"
    num_threads=4,  # Adjust based on your system
    verbose=True
)

print("⚙️ MIPROv2 Optimizer configured:")
print("   - Metric: custom_metric (F1 >= 0.5)")
print("   - Mode: light (faster, good for demos)")
print("   - Threads: 4")
print("\n💡 Optimization will:")
print("   1. Bootstrap few-shot examples from training data")
print("   2. Generate instruction candidates")
print("   3. Evaluate candidate programs on validation set")
print("   4. Select best combination using Bayesian optimization")


⚙️ MIPROv2 Optimizer configured:
   - Metric: custom_metric (F1 >= 0.5)
   - Mode: light (faster, good for demos)
   - Threads: 4

💡 Optimization will:
   1. Bootstrap few-shot examples from training data
   2. Generate instruction candidates
   3. Evaluate candidate programs on validation set
   4. Select best combination using Bayesian optimization


In [10]:
print("🚀 Starting optimization process...")
print("⏳ This may take 2-5 minutes depending on dataset size\n")

# Compile optimized program
optimized_agent = optimizer.compile(
    student=react_agent,
    trainset=trainset,
    valset=valset,
    requires_permission_to_run=False  # Skip permission prompt
)

print("\n✅ Optimization complete!")


2026/02/04 14:22:47 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2026/02/04 14:22:47 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3bb1812

🚀 Starting optimization process...
⏳ This may take 2-5 minutes depending on dataset size

Bootstrapping set 1/3
Bootstrapping set 2/3
Bootstrapping set 3/3


  0%|          | 0/7 [00:00<?, ?it/s]d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## re...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
 14%|█▍        | 1/7 [00:00<00:05,  1.03it/s]d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='messa

Bootstrapped 4 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.


2026/02/04 14:22:56 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2026/02/04 14:22:56 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.


...

d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## ob...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(
d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## su...er_specific_fields=None), input_type=

DATA SUMMARY: The dataset consists of diverse question-answer pairs covering topics such as programming, history, science, and technology, with a focus on factual and concise questions. The straightforward syntax and broad range of domains suggest that the dataset could be used to train a general knowledge or trivia question-answering system. The model trained on this data could potentially handle a wide range of inquiries across various subjects.
Using a randomly generated configuration for our grounded proposer.
Selected tip: creative


d:\User\Desktop\Acads\4th Year\TwoTabs Dir\.venv\Lib\site-packages\pydantic\main.py:528: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 10 fields but got 5: Expected `Message` - serialized value may not be as expected [field_name='message', input_value=Message(content='[[ ## pr...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [field_name='choices', input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_json(


PROGRAM DESCRIPTION: The provided program appears to be designed to solve tasks that involve answering questions by interleaving thought processes with the use of various tools, specifically a Wikipedia search tool and a finish tool. The program is structured around a reactive approach, where it iteratively generates the next thought, selects the next tool to use, and provides arguments for that tool based on the current state of the task. The program continues this process until it decides to use the "finish" tool, at which point it extracts the final answer based on the trajectory of thoughts, tool usage, and observations made during the task-solving process.

The program utilizes a modular design, incorporating components such as a `ReAct` module that manages the reactive process, and tools like `search_wikipedia` that can be used to gather information relevant to the task at hand. The `ReAct` module is initialized with a signature that defines the input and output fields for the ta

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01j7fskkkre6c8m8d7kk10szk5` service tier `on_demand` on tokens per minute (TPM): Limit 12000, Used 9987, Requested 2773. Please try again in 3.8s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
